# Dimensionality Reduction 3

Learning objectives
1. Apply t-SNE and UMAP to different data sets and interpret the outputs
2. Learn how to visualise the model outputs
3. Explore differences based on chosen parameters
4. Interpret the results and compare them to other dimension reduction methods

UMAP is implemented in the `umap-learn` package

Install it via ```pip install umap-learn```

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import umap

## Load in datasets
Read in the omics datasets using the pandas [read_excel()](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) function. For this example we will be using IBD metagenomic data, summarised at the species level.

In [ ]:
IBD_microbiome = pd.read_excel("../Data/IBD_microbiome_species.xlsx")

In [ ]:
IBD_microbiome.head()

We can see the data contains 4 metadata columns: gender, age, inflammatory bowel disease (IBD) status, and IBD type. 

In [ ]:
# look at the number of IBD vs non-IBD patients
IBD_microbiome["IBD"].value_counts()

### Scaling, exploratory analysis using PCA, and outlier detection

In [ ]:
# Standard scale the data to have mean = 0 and SD = 1
IBD_microbiome_scaled = StandardScaler().fit_transform(IBD_microbiome.iloc[:, 4:])

In [ ]:
# perform PCA to perliminarily visualise the data and detect outliers
PCA_IBD = PCA(n_components=4).fit_transform(IBD_microbiome_scaled)

In [ ]:
PCA_df = pd.DataFrame(PCA_IBD, index=IBD_microbiome.index)
PCA_df["IBD_type"] = IBD_microbiome["IBD_type"]
PCA_df

In [ ]:
# plot the first 4 PCA components against each other
p = sns.pairplot(PCA_df, hue="IBD_type")
plt.show()

There are a number of extreme outliers we must remove here. To do this, we will visually inspect the PCA scores and remove those samples with very high absolute values across components 1-4. 

In [ ]:
# Loop through each PCA component scores
for i in range(0 ,4):
    print("Component " + str(i+1))
    # order the scores from highest to lowest absolute value and print the top 10
    # the row index (sample) is the dictionary key, and the PC score is the value
    print(PCA_df[i].sort_values(key=abs, ascending=False)[0:10].to_dict())

The outliers are present in samples (row indexes) 260, 339, 346, 312, and 165. We will drop these from the data:

In [ ]:
# drop the row indexes containing outlier samples
IBD_microbiome = IBD_microbiome.drop([260, 339, 346, 312, 165])

In [ ]:
IBD_microbiome_scaled = StandardScaler().fit_transform(IBD_microbiome.iloc[:, 4:])
PCA_IBD = PCA(n_components=4).fit_transform(IBD_microbiome_scaled)
PCA_df = pd.DataFrame(PCA_IBD, index=IBD_microbiome.index)
PCA_df["IBD_type"] = IBD_microbiome["IBD_type"]

# plot the first 4 PCA components against each other
p = sns.pairplot(PCA_df, hue="IBD_type")
plt.show()

The data looks a bit better but there are still some extreme outliers, let's repeat the outlier detection process again:

In [ ]:
for i in range(0 ,4):
    print(PCA_df[i].sort_values(key=abs, ascending=False)[0:10].to_dict())

In [ ]:
IBD_microbiome = IBD_microbiome.drop([336, 335, 208, 369, 199, 315, 299])

In [ ]:
IBD_microbiome_scaled = StandardScaler().fit_transform(IBD_microbiome.iloc[:, 4:])
PCA_IBD = PCA(n_components=4).fit_transform(IBD_microbiome_scaled)
PCA_df = pd.DataFrame(PCA_IBD, index=IBD_microbiome.index)
PCA_df["IBD_type"] = IBD_microbiome["IBD_type"]

# plot the first 4 PCA components against each other
p = sns.pairplot(PCA_df, hue="IBD_type")
plt.show()

The extreme outliers have now been removed, so we can move on to appling t-SNE

## T-distributed Stochastic Neighbor Embedding (t-SNE)
t-SNE is a tool to visualize high-dimensional data. It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. t-SNE has a cost function that is not convex, i.e. with different initializations we can get different results.

t-SNE can be perfomed using the sklearn.[TSNE()](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) function. We apply the fit_transform function to the TSNE class to create an embedding of the metagenomic data.

In [ ]:
# perfrom t-SNE with default parameters
tsne_embedding = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(IBD_microbiome_scaled)

In [ ]:
sns.set_style("ticks")
sns.set_context("notebook")
sns.set_palette("Paired")
plt.figure(figsize=(8, 8))

p = sns.scatterplot(x=tsne_embedding[:, 0],
 y=tsne_embedding[:, 1], 
 hue=IBD_microbiome["IBD_type"])

p.set_xlabel("Dim 1")
p.set_ylabel("Dim 2")

plt.show()

### t-SNE parameters

There are 4 key [parameters](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) that need tuning when performing t-SNE. These are [taken from sklearn]:
1. `n_components`: similar to PCA, NNMF, and MDS, we need to manually specify the number of components
2. `perplexity`: The perplexity is related to the number of nearest neighbors that is used in other manifold learning algorithms. Larger datasets usually require a larger perplexity. Consider selecting a value between 5 and 50. Different values can result in significantly different results.
3. `early_exaggeration`: Controls how tight natural clusters in the original space are in the embedded space and how much space will be between them. For larger values, the space between natural clusters will be larger in the embedded space. Again, the choice of this parameter is not very critical. If the cost function increases during initial optimization, the early exaggeration factor or the learning rate might be too high.
4. `learning_rate`: The learning rate for t-SNE is usually in the range [10.0, 1000.0]. If the learning rate is too high, the data may look like a ‘ball’ with any point approximately equidistant from its nearest neighbours. If the learning rate is too low, most points may look compressed in a dense cloud with few outliers. If the cost function gets stuck in a bad local minimum increasing the learning rate may help.

Let's test some of these parameters and see what effects they have on the visualisation of the data:

#### Perplexity - n neighbours

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

axs = [ax1, ax2, ax3]

# set some perplexity values we want to test
perplexity_vals = [2, 25, 50]

# loop over each perplexity value and perform t-SNE with this value 
for ax, i in enumerate(perplexity_vals):
    tsne_embedding = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=i).fit_transform(IBD_microbiome_scaled)    

    # plot the first two components using a scatterplot
    sns.scatterplot(x=tsne_embedding[:, 0], y=tsne_embedding[:, 1], hue=IBD_microbiome["IBD_type"], ax=axs[ax])
    axs[ax].set_title("Perplexity="+str(i))

plt.tight_layout()
plt.show()

#### Learning rate

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15, 5))

axs = [ax1, ax2, ax3, ax4]

# set some  values we want to test
learningrate_vals = [50, 100, 800, 1000]

# loop over each value and perform t-SNE with this value 
for ax, i in enumerate(learningrate_vals):
    tsne_embedding = TSNE(n_components=2, init='random', learning_rate=i).fit_transform(IBD_microbiome_scaled)    

    # plot the first two components using a scatterplot
    sns.scatterplot(x=tsne_embedding[:, 0], y=tsne_embedding[:, 1], hue=IBD_microbiome["IBD_type"], ax=axs[ax])
    axs[ax].set_title("Learning rate="+str(i))

plt.tight_layout()
plt.show()

Have a go at changing the learning rate and perplexity parameters to visualise the data in the scatter plot below. What is the optimal combination of parameters you can find?

Note: unlike PCA, in t-SNE there are multiple solutions, so the output will be different each time

In [ ]:
tsne_embedding = TSNE(n_components=2, init='random', learning_rate=500, perplexity=20).fit_transform(IBD_microbiome_scaled)    

plt.figure(figsize=(8, 8))
p = sns.scatterplot(x=tsne_embedding[:, 0],
 y=tsne_embedding[:, 1], 
 hue=IBD_microbiome["IBD_type"], 
 s=150, 
 alpha=0.7)

p.set_xlabel("Dim 1")
p.set_ylabel("Dim 2")
plt.title("t-SNE of IBD microbiome data")

plt.show()

### Using PCA to reduce dimensionality prior to t-SNE
It is highly recommended to run PCA before t-SNE to reduce computation time, particularly on very high dimensional datasets. 30-100 components is ideal. 

In [ ]:
# Run 50 component PCA on the scaled data
PCA_IBD = PCA(n_components=50).fit_transform(IBD_microbiome_scaled)

plt.figure(figsize=(6, 6))
p = sns.scatterplot(x=PCA_IBD[:, 0],
 y=PCA_IBD[:, 1], 
 hue=IBD_microbiome["IBD_type"])

p.set_xlabel("PC1")
p.set_ylabel("PC2")
plt.show()

In [ ]:
# make a dataframe with the 50 PCA components we have just computed
PCA_IBD_df = pd.DataFrame(PCA_IBD)
PCA_IBD_df

In [ ]:
# use the 50 PCA components as input for tSNE
tsne_embedding_PCA = TSNE(n_components=2, init='random', learning_rate=500, perplexity=25).fit_transform(PCA_IBD_df)    

# plot the tSNE components - colour using different covariates
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

sns.scatterplot(x=tsne_embedding_PCA[:, 0], y=tsne_embedding_PCA[:, 1], hue=IBD_microbiome["IBD_type"], ax=ax1)
ax1.set_title("IBD status")

sns.scatterplot(x=tsne_embedding_PCA[:, 0], y=tsne_embedding_PCA[:, 1], hue=IBD_microbiome["Gender"], ax=ax2)
ax2.set_title("Gender")

IBD_microbiome['Age'] = IBD_microbiome['Age'].replace("na", np.nan)
sns.scatterplot(x=tsne_embedding_PCA[:, 0], y=tsne_embedding_PCA[:, 1], hue=IBD_microbiome["Age"], ax=ax3, palette="viridis")
ax3.set_title("Age")

plt.tight_layout()
plt.show()

## Uniform Manifold Approximation & Projection (UMAP)
Read the UMAP [documentation](https://umap-learn.readthedocs.io/en/latest/index.html) for an in-depth description of the method and the implementation. In the `umap-learn` package, UMAP is implemented via the `umap.UMAP()` class. Just like in sklearn, we use the `fit_transform()` function to project the embedding onto the data.

In [ ]:
# perform UMAP and project onto the scaled IBD data
UMAP_embedding = umap.UMAP().fit_transform(IBD_microbiome_scaled)

In [ ]:
# plot UMAP embedding
plt.figure(figsize=(8, 8))
p = sns.scatterplot(x=UMAP_embedding[:, 0],
 y=UMAP_embedding[:, 1], 
 hue=IBD_microbiome["IBD_type"], 
 s=150, 
 alpha=0.7)

p.set_xlabel("Dim 1")
p.set_ylabel("Dim 2")
plt.title("UMAP embedding of IBD microbiome data")

plt.show()

### UMAP parameters
There are 4 key [parameters](https://umap-learn.readthedocs.io/en/latest/parameters.html) in UMAP (taken from UMAP website):
1. `n_components`: Same as PCA, t-SNE, etc, allows you to decide the number of components to reduce the input data to.
2. `n_neighbours`: This parameter controls how UMAP balances local versus global structure in the data. It does this by constraining the size of the local neighborhood UMAP will look at when attempting to learn the manifold structure of the data. This means that low values of n_neighbors will force UMAP to concentrate on very local structure (potentially to the detriment of the big picture), while large values will push UMAP to look at larger neighborhoods of each point when estimating the manifold structure of the data, losing fine detail structure for the sake of getting the broader of the data.
3. `min_dist`: The min_dist parameter controls how tightly UMAP is allowed to pack points together. It, quite literally, provides the minimum distance apart that points are allowed to be in the low dimensional representation. This means that low values of min_dist will result in clumpier embeddings. This can be useful if you are interested in clustering, or in finer topological structure. Larger values of min_dist will prevent UMAP from packing points together and will focus on the preservation of the broad topological structure instead.
4. `metric`: This controls how distance is computed in the ambient space of the input data. By default UMAP supports a wide variety of metrics, including Euclidean distance (see full [list](https://umap-learn.readthedocs.io/en/latest/parameters.html#metric)) as well as custom metrics. 

Let's look at these parameters in more detail:

#### n_neighbours

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15, 5))

axs = [ax1, ax2, ax3, ax4]

# set some  values we want to test
n_neighbours_vals = [2, 25, 50, 200]

# loop over each value and perform t-SNE with this value 
for ax, i in enumerate(n_neighbours_vals):
    UMAP_embedding = umap.UMAP(n_components=2, n_neighbors=i).fit_transform(IBD_microbiome_scaled)

    # plot the first two components using a scatterplot
    sns.scatterplot(x=UMAP_embedding[:, 0], y=UMAP_embedding[:, 1], hue=IBD_microbiome["IBD_type"], ax=axs[ax])
    axs[ax].set_title("n_neighbours="+str(i))

plt.tight_layout()
plt.show()

#### min_dist

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15, 5))

axs = [ax1, ax2, ax3, ax4]

# set some  values we want to test
min_dist_vals = [0, 0.1, 0.5, 0.99]

# loop over each value and perform t-SNE with this value 
for ax, i in enumerate(min_dist_vals):
    UMAP_embedding = umap.UMAP(n_components=2, min_dist=i).fit_transform(IBD_microbiome_scaled)

    # plot the first two components using a scatterplot
    sns.scatterplot(x=UMAP_embedding[:, 0], y=UMAP_embedding[:, 1], hue=IBD_microbiome["IBD_type"], ax=axs[ax])
    axs[ax].set_title("min_dist="+str(i))

plt.tight_layout()
plt.show()

#### metric

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

axs = [ax1, ax2, ax3]

# set some  values we want to test
metric_vals = ["euclidean", "manhattan", "minkowski"]

# loop over each value and perform t-SNE with this value 
for ax, i in enumerate(metric_vals):
    UMAP_embedding = umap.UMAP(n_components=2, metric=i).fit_transform(IBD_microbiome_scaled)

    # plot the first two components using a scatterplot
    sns.scatterplot(x=UMAP_embedding[:, 0], y=UMAP_embedding[:, 1], hue=IBD_microbiome["IBD_type"], ax=axs[ax])
    axs[ax].set_title("metric="+str(i))

plt.tight_layout()
plt.show()

Try changing these parameters to obtain the best possible embedding on the data:

You can also try colouring the plot by other covariates such as Age, or Gender, by changing the `hue` parameter in the plotting function.

In [ ]:
# change these parameters:
UMAP_embedding = umap.UMAP(n_components=2, metric="minkowski", n_neighbors=4, min_dist=0.7).fit_transform(IBD_microbiome_scaled)

plt.figure(figsize=(8, 8))
p = sns.scatterplot(x=UMAP_embedding[:, 0],
 y=UMAP_embedding[:, 1], 
 hue=IBD_microbiome["IBD_type"], # change this to colour points by other covariates
 s=150, 
 alpha=0.7)

p.set_xlabel("Dim 1")
p.set_ylabel("Dim 2")

plt.show()

## Your turn

Load in another dataset in the `Data` folder  and perform PCA, t-SNE and UMAP on it, visusalising the data using scatterplots as above. Tune the parameters as necessary. Which dimensionality reduction approach would you use for visualisation?

In [ ]:
# Import dataset

In [ ]:
# Scale the data

In [ ]:
# Perform PCA

# Visualise PCA

In [ ]:
# Perform t-SNE

# Visualise t-SNE

In [ ]:
# Perform UMAP

# Visualise UMAP